In [5]:
import numpy as np
import os
from obspy.clients.iris import Client
import obspy.geodetics.base as base 
stafile="./Input/Stations.dat"
evefile="./Input/EventCatlog"
tstarPfile="./Input/tstar_P_1.28sSNR3.dat"
tstarSfile="./Input/tstar_S_1.28sSNR3.dat"
phasefile="./Input/phase.dat"

with open(phasefile,"r") as f:
    temp=f.read().splitlines()
    TravelTime={}
    for line in temp:
        para=line.split()
        if para[0]=="#":
            evid=para[1]
        else:
            staname,t,qual,phase=para
            t=float(t)
            key="_".join([evid,staname,phase])
            TravelTime[key]=t

with open(stafile,"r") as f:
    temp=f.read().splitlines()
    STA={}
    for line in temp:
        para=line.split()
        staname,lat,lon,dep=para
        STA[staname]=[float(lon),float(lat),-float(dep)/1000]
with open(evefile,"r") as f:
    temp=f.read().splitlines()
    EVE={}
    for line in temp:
        para=line.split()
        time,evid,lon,lat,dep,mag=para[0],para[9],para[3],para[2],para[4],para[5]
        EVE[evid]=list(map(float,[lon,lat,dep,mag,time]))

def readTstarInformation(tstarfile,EVE,STA,TravelTime,phaseflag):
    with open(tstarfile,"r") as f:
        temp=f.read().splitlines()
        tstar_dist={}
        for line in temp:
            para=line.split()
            if para[0]=="#":
                evid=para[1]
                evelon,evelat,evedep,mag,time=EVE[evid]
            else:
                staname=para[0]
                stalon,stalat,stadep=STA[staname]
                tstar=float(para[1])
                fit=para[2]
                phase=para[3]
                dlat=stalat-evelat;dlon=stalon-evelon
                dist=np.sqrt((dlat*111)**2 + (dlon*(np.cos(evelat*3.1415926/180)*111))**2 + (evedep-stadep)**2 ) #
#                 result = Client().distaz(stalat,stalon,  evelat,evelon)
#                 baz=result['backazimuth'];
                dis,az,baz=base.gps2dist_azimuth(stalat,stalon,evelat,evelon)
                key="_".join([evid,staname,phaseflag])
                travelTime= TravelTime[key]
                if tstar<=0.00001:
                    Q=np.inf 
                else:
                    Q=travelTime/tstar   
            
                if staname in tstar_dist:
                    tstar_dist[staname].append([tstar,dist,evedep,baz,evelon,evelat,mag,time,Q])
                else:
                    tstar_dist[staname]=[]
                    tstar_dist[staname].append([tstar,dist,evedep,baz,evelon,evelat,mag,time,Q])
    return tstar_dist

def SelectTstar(tstarfile,EVE,STA,TravelTime,Qlim,phaseflag,outfile):
    fid=open(outfile,"w")
    with open(tstarfile,"r") as f:
        temp=f.read().splitlines()
        tstar_dist={}
        for line in temp:
            para=line.split()
            if para[0]=="#":
                evid=para[1]
                evelon,evelat,evedep,mag,time=EVE[evid]
                fid.write("#  {}\n".format(evid))
            else:
                staname=para[0]
                stalon,stalat,stadep=STA[staname]
                tstar=float(para[1])
                fit=para[2]
                phase=para[3]
                key="_".join([evid,staname,phaseflag])
                travelTime= TravelTime[key]
                if tstar<=0.00000001:
                    Q=99999999999999
                else:
                    Q=travelTime/tstar
                if Q<Qlim:
                    fid.write("{} {:.5f} {} {}\n".format(staname,tstar,fit,phase))
        fid.close()

In [8]:
os.system("rm -rf Output && mkdir Output")
Qlim=1500
outfileP="./Output/P_tstar_selected_temp.dat"
outfileS="./Output/S_tstar_selected_temp.dat"
SelectTstar(tstarPfile,EVE,STA,TravelTime,Qlim,"P",outfileP)
SelectTstar(tstarSfile,EVE,STA,TravelTime,Qlim,"S",outfileS)
os.system("awk -f delete_no_picks_events.awk Output/P_tstar_selected_temp.dat >Output/P_tstar_selected.dat")
os.system("awk -f delete_no_picks_events.awk Output/S_tstar_selected_temp.dat >Output/S_tstar_selected.dat")
os.system("cd Output/ && rm -rf *temp*")

0